# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 14 2022 12:52PM,250711,10,8774771,"Snap Medical Industries, LLC",Released
1,Nov 14 2022 12:52PM,250711,10,8774772,"Snap Medical Industries, LLC",Released
2,Nov 14 2022 12:52PM,250711,10,8774773,"Snap Medical Industries, LLC",Released
3,Nov 14 2022 12:52PM,250711,10,8774774,"Snap Medical Industries, LLC",Released
4,Nov 14 2022 12:52PM,250711,10,8774790,"Snap Medical Industries, LLC",Released
5,Nov 14 2022 12:52PM,250711,10,8774791,"Snap Medical Industries, LLC",Released
6,Nov 14 2022 12:52PM,250711,10,8775292,"Snap Medical Industries, LLC",Released
7,Nov 14 2022 12:52PM,250711,10,8775293,"Snap Medical Industries, LLC",Released
8,Nov 14 2022 12:52PM,250711,10,8775294,"Snap Medical Industries, LLC",Released
9,Nov 14 2022 12:52PM,250711,10,8775295,"Snap Medical Industries, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,250707,Released,1
33,250708,Released,10
34,250709,Released,23
35,250710,Released,1
36,250711,Released,11


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
250707,NaN,NaN,1.0
250708,NaN,NaN,10.0
250709,NaN,NaN,23.0
250710,NaN,NaN,1.0
250711,NaN,NaN,11.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250634,0.0,0.0,2.0
250638,0.0,0.0,5.0
250664,4.0,13.0,1.0
250668,0.0,6.0,31.0
250669,0.0,0.0,4.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250634,0,0,2
250638,0,0,5
250664,4,13,1
250668,0,6,31
250669,0,0,4


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250634,0,0,2
1,250638,0,0,5
2,250664,4,13,1
3,250668,0,6,31
4,250669,0,0,4


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250634,,,2
1,250638,,,5
2,250664,4,13,1
3,250668,,6,31
4,250669,,,4


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 14 2022 12:52PM,250711,10,"Snap Medical Industries, LLC"
11,Nov 14 2022 12:51PM,250710,10,Yusen – 3D Matrix
12,Nov 14 2022 12:44PM,250709,10,ISDIN Corporation
35,Nov 14 2022 12:41PM,250708,10,ISDIN Corporation
45,Nov 14 2022 12:41PM,250704,10,ISDIN Corporation
106,Nov 14 2022 12:41PM,250706,10,ISDIN Corporation
124,Nov 14 2022 12:40PM,250700,10,ISDIN Corporation
180,Nov 14 2022 12:38PM,250707,10,Yusen – 3D Matrix
181,Nov 14 2022 12:36PM,250703,19,"Innogenix, LLC"
205,Nov 14 2022 12:35PM,250705,10,Yusen – 3D Matrix


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Nov 14 2022 12:52PM,250711,10,"Snap Medical Industries, LLC",,,11
1,Nov 14 2022 12:51PM,250710,10,Yusen – 3D Matrix,,,1
2,Nov 14 2022 12:44PM,250709,10,ISDIN Corporation,,,23
3,Nov 14 2022 12:41PM,250708,10,ISDIN Corporation,,,10
4,Nov 14 2022 12:41PM,250704,10,ISDIN Corporation,,,61
5,Nov 14 2022 12:41PM,250706,10,ISDIN Corporation,,,18
6,Nov 14 2022 12:40PM,250700,10,ISDIN Corporation,,,56
7,Nov 14 2022 12:38PM,250707,10,Yusen – 3D Matrix,,,1
8,Nov 14 2022 12:36PM,250703,19,"Innogenix, LLC",,,24
9,Nov 14 2022 12:35PM,250705,10,Yusen – 3D Matrix,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 14 2022 12:52PM,250711,10,"Snap Medical Industries, LLC",11,,
1,Nov 14 2022 12:51PM,250710,10,Yusen – 3D Matrix,1,,
2,Nov 14 2022 12:44PM,250709,10,ISDIN Corporation,23,,
3,Nov 14 2022 12:41PM,250708,10,ISDIN Corporation,10,,
4,Nov 14 2022 12:41PM,250704,10,ISDIN Corporation,61,,
5,Nov 14 2022 12:41PM,250706,10,ISDIN Corporation,18,,
6,Nov 14 2022 12:40PM,250700,10,ISDIN Corporation,56,,
7,Nov 14 2022 12:38PM,250707,10,Yusen – 3D Matrix,1,,
8,Nov 14 2022 12:36PM,250703,19,"Innogenix, LLC",24,,
9,Nov 14 2022 12:35PM,250705,10,Yusen – 3D Matrix,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 14 2022 12:52PM,250711,10,"Snap Medical Industries, LLC",11,,
1,Nov 14 2022 12:51PM,250710,10,Yusen – 3D Matrix,1,,
2,Nov 14 2022 12:44PM,250709,10,ISDIN Corporation,23,,
3,Nov 14 2022 12:41PM,250708,10,ISDIN Corporation,10,,
4,Nov 14 2022 12:41PM,250704,10,ISDIN Corporation,61,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 14 2022 12:52PM,250711,10,"Snap Medical Industries, LLC",11.0,NaN,NaN
1,Nov 14 2022 12:51PM,250710,10,Yusen – 3D Matrix,1.0,NaN,NaN
2,Nov 14 2022 12:44PM,250709,10,ISDIN Corporation,23.0,NaN,NaN
3,Nov 14 2022 12:41PM,250708,10,ISDIN Corporation,10.0,NaN,NaN
4,Nov 14 2022 12:41PM,250704,10,ISDIN Corporation,61.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 14 2022 12:52PM,250711,10,"Snap Medical Industries, LLC",11.0,0.0,0.0
1,Nov 14 2022 12:51PM,250710,10,Yusen – 3D Matrix,1.0,0.0,0.0
2,Nov 14 2022 12:44PM,250709,10,ISDIN Corporation,23.0,0.0,0.0
3,Nov 14 2022 12:41PM,250708,10,ISDIN Corporation,10.0,0.0,0.0
4,Nov 14 2022 12:41PM,250704,10,ISDIN Corporation,61.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5013809,525.0,23.0,0.0
12,501394,0.0,2.0,0.0
15,752007,7.0,14.0,6.0
19,501372,28.0,0.0,0.0
20,250695,1.0,0.0,0.0
21,501347,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5013809,525.0,23.0,0.0
1,12,501394,0.0,2.0,0.0
2,15,752007,7.0,14.0,6.0
3,19,501372,28.0,0.0,0.0
4,20,250695,1.0,0.0,0.0
5,21,501347,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,525.0,23.0,0.0
1,12,0.0,2.0,0.0
2,15,7.0,14.0,6.0
3,19,28.0,0.0,0.0
4,20,1.0,0.0,0.0
5,21,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,525.0
1,12,Released,0.0
2,15,Released,7.0
3,19,Released,28.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21
Status,,,,,,
Completed,0.0,0.0,6.0,0.0,0.0,0.0
Executing,23.0,2.0,14.0,0.0,0.0,0.0
Released,525.0,0.0,7.0,28.0,1.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21
0,Completed,0.0,0.0,6.0,0.0,0.0,0.0
1,Executing,23.0,2.0,14.0,0.0,0.0,0.0
2,Released,525.0,0.0,7.0,28.0,1.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21
0,Completed,0.0,0.0,6.0,0.0,0.0,0.0
1,Executing,23.0,2.0,14.0,0.0,0.0,0.0
2,Released,525.0,0.0,7.0,28.0,1.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()